### Sort Filters

In [1]:
import glob 
import astropy.io.fits as pf
import numpy as np

# Sort flats into filters
Flat = glob.glob("C:/Users/dirmu/Documents/fieldtrip/IAC80/17mar20/Flat/*.fits")
Bias = glob.glob("C:/Users/dirmu/Documents/fieldtrip/IAC80/17mar20/Bias/*.fits")
O45P = glob.glob("C:/Users/dirmu/Documents/fieldtrip/IAC80/17mar20/45P/*.fits")
O71P = glob.glob("C:/Users/dirmu/Documents/fieldtrip/IAC80/17mar20/71P/*.fits")

V_f = []
R_f = []
I_f = []

for name in Flat:
    O = pf.open(name)
    H = O[0].header
#     if H["SPEED"] == 500:
    if H["INSFILTE"] == "V":
        V_f.append(name)
    elif H["INSFILTE"] == "R":
        R_f.append(name)
    elif H["INSFILTE"] == "I":
        I_f.append(name)
    else:
        pass
# else:
#     pass
    
# Sort objects into filters
V_o = []
R_o = []
I_o = []

for name in O71P:
    O = pf.open(name)
    H = O[0].header
    if H["INSFILTE"] == "V":
        V_o.append(name)
    elif H["INSFILTE"] == "R":
        R_o.append(name)
    elif H["INSFILTE"] == "I":
        I_o.append(name)
    else:
        pass

# myBias = []
# # Sort Bias, only take 500Hz
# for name in Bias:
#     O = pf.open(name)
#     H = O[0].header
#     if H["SPEED"] == 500:
#         myBias.append(name)
#     else:
#         pass

### Mean Combine Flats and Bias'

In [2]:
def mcomb(file_loc,file_type):
    file = file_loc
    # Empty list for storing trimmed flats
    A = []
    for name in file:
        # Open file
        O = pf.open(name)
        # Acess image data
        D = O[0].data
        # Trim
        D = D[0:2048,50:2098]
        A.append(D)
        
    medflat = np.median(A, axis=0)
    if "flat" in file_type:
        masterflat = medflat/np.mean(A)
        hdu = pf.PrimaryHDU(masterflat)
    else:
        hdu = pf.PrimaryHDU(medflat)
    hdu.writeto('medcomb'+file_type+'.fits')
    
mcomb(V_f,'flat_v')
mcomb(R_f,'flat_r')
mcomb(I_f,'flat_i')


mcomb(Bias,'bias')

In [3]:
def reduce(obj_loc,filte):
    # List of object files
    file = obj_loc
    flat = pf.open("medcombflat_"+filte+".fits")
    flat_d = flat[0].data
    bias = pf.open("medcombbias.fits")
    bias_d = bias[0].data
    for name in file:
        ob = pf.open(name)
        ob_d = ob[0].data
        ob_h = ob[0].header
        
        # Trim
        ob_d = ob_d[0:2048,50:2098]
        del(ob_h["camtemp"])
        del(ob_h["pressure"])
        
        # Reduce image
        # And save as 8Mb
        flat_d = flat_d - bias_d
        ob_d = (ob_d - bias_d)/flat_d
        ob_d = np.array(ob_d,dtype=np.int16)
        ob_d = abs(ob_d)
        # Take filepath string for filename
        # This will need to be altered
        file_name = name[-17:]
        hdu = pf.PrimaryHDU(ob_d,ob_h)
        hdu.writeto("red_"+file_name)
        
reduce(V_o,"v")
reduce(R_o,"r")
reduce(I_o,"i")